In [ ]:
!pip install tqdm scidownl

In [1]:
from scidownl import scihub_download
import pandas as pd
from tqdm import tqdm
import concurrent.futures

In [ ]:
def download_papers(paper):
    name = paper.replace('/', ':')
    scihub_download(paper, paper_type="doi", out=f"./papers/{name}.pdf")

df = pd.read_csv('output.csv')
source = list(df['DOI'])
source = [doi for doi in source if pd.notna(doi)]

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(download_papers, paper) for paper in source]

    # Iterate over completed futures to check progress
    for _ in tqdm(concurrent.futures.as_completed(futures), total=len(source)):
        pass

import os

folder_path = "papers"  # Replace "papers" with your folder path

# Get the list of files in the folder
file_list = os.listdir(folder_path)

# Count the number of files
num_files = len(file_list)

print(f"The number of files in the folder '{folder_path}' is: {num_files} out of {len(source)} initial DOIs")

[INFO] | 2023/07/12 17:45:29 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2023/07/12 17:45:29 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2023/07/12 17:45:29 | Choose scihub url [0]: https://sci-hub.se


[INFO] | 2023/07/12 17:45:29 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2023/07/12 17:45:29 | Choose scihub url [0]: https://sci-hub.se
  0%|          | 0/301 [00:00<?, ?it/s][INFO] | 2023/07/12 17:45:29 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1016/j.apsoil.2023.104931], proxies={}
[INFO] | 2023/07/12 17:45:29 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1016/j.rsase.2023.100930], proxies={}
[INFO] | 2023/07/12 17:45:29 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1016/j.scitotenv.2023.163226], proxies={}
[INFO] | 2023/07/12 17:45:29 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.3390/agriculture13030704], proxies={}
[INFO] | 2023/07/12 17:45:29 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1016/j.envres.2023.115960], proxies={}
[INFO] | 2023/07/12 17:45:30 | -> Response: status_code=200, content_length=5836
[WA

100% [==================================================] 146/146


[INFO] | 2023/07/12 17:45:44 | ↓ Successfully download the url to: ./papers/10.1038:s41598-022-15802-6.pdf
  5%|▍         | 15/301 [00:14<03:26,  1.38it/s][INFO] | 2023/07/12 17:45:44 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2023/07/12 17:45:44 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1016/j.jhydrol.2022.128548], proxies={}


100% [==================================================] 146/146


...

 98% [================================================= ] 8101888/8236245

[INFO] | 2023/07/12 17:46:14 | -> Response: status_code=403, content_length=11541
[WARNING] | 2023/07/12 17:46:14 | Error occurs, task status: crawling_failed, error: Error occurs when crawling source: DoiSource[type=doi, id=10.11733/j.issn.1007-0435.2021.04.015]
[INFO] | 2023/07/12 17:46:14 | Choose scihub url [1]: http://sci-hub.st
[INFO] | 2023/07/12 17:46:14 | <- Request: scihub_url=http://sci-hub.st, source=DoiSource[type=doi, id=10.11733/j.issn.1007-0435.2021.04.015], proxies={}
[INFO] | 2023/07/12 17:46:14 | -> Response: status_code=403, content_length=11541
[WARNING] | 2023/07/12 17:46:14 | Error occurs, task status: crawling_failed, error: Error occurs when crawling source: DoiSource[type=doi, id=10.13522/j.cnki.ggps.2020587]
[INFO] | 2023/07/12 17:46:14 | Choose scihub url [3]: http://sci-hub.se
[INFO] | 2023/07/12 17:46:14 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.13522/j.cnki.ggps.2020587], proxies={}


100% [==================================================] 8236245/8236245


[INFO] | 2023/07/12 17:46:14 | ↓ Successfully download the url to: ./papers/10.3390:w13121677.pdf
 22%|██▏       | 66/301 [00:44<01:23,  2.81it/s][INFO] | 2023/07/12 17:46:14 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2023/07/12 17:46:14 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1088/1755-1315/663/1/012026], proxies={}
[INFO] | 2023/07/12 17:46:14 | -> Response: status_code=403, content_length=11541
[WARNING] | 2023/07/12 17:46:14 | Error occurs, task status: crawling_failed, error: Error occurs when crawling source: DoiSource[type=doi, id=10.3390/f12030270]
[INFO] | 2023/07/12 17:46:14 | Choose scihub url [1]: http://sci-hub.st
[INFO] | 2023/07/12 17:46:14 | <- Request: scihub_url=http://sci-hub.st, source=DoiSource[type=doi, id=10.3390/f12030270], proxies={}
[INFO] | 2023/07/12 17:46:14 | -> Response: status_code=403, content_length=11541
[WARNING] | 2023/07/12 17:46:14 | Error occurs, task status: crawling_failed, error: Error occurs whe

[INFO] | 2023/07/12 17:47:39 | -> Response: status_code=403, content_length=11541
[WARNING] | 2023/07/12 17:47:39 | Error occurs, task status: crawling_failed, error: Error occurs when crawling source: DoiSource[type=doi, id=10.1016/S0341-8162(02)00093-0]
[INFO] | 2023/07/12 17:47:39 | Choose scihub url [1]: http://sci-hub.st
[INFO] | 2023/07/12 17:47:39 | <- Request: scihub_url=http://sci-hub.st, source=DoiSource[type=doi, id=10.1016/S0341-8162(02)00093-0], proxies={}
[INFO] | 2023/07/12 17:47:39 | -> Response: status_code=403, content_length=11541
[WARNING] | 2023/07/12 17:47:40 | Error occurs, task status: crawling_failed, error: Error occurs when crawling source: DoiSource[type=doi, id=10.1016/S0016-7061(02)00226-4]
[INFO] | 2023/07/12 17:47:40 | Choose scihub url [1]: http://sci-hub.st
[INFO] | 2023/07/12 17:47:40 | <- Request: scihub_url=http://sci-hub.st, source=DoiSource[type=doi, id=10.1016/S0016-7061(02)00226-4], proxies={}
[INFO] | 2023/07/12 17:47:40 | -> Response: status_c

#### PDF Downloader

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm


In [10]:
df = pd.read_csv('output.csv')
source = list(df['DOI'])
print(len(source))

source = [doi for doi in source if pd.notna(doi)]
print(len(source))

350
301


In [42]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from itertools import cycle

titles = []
citations = []
dois = []

import requests
import pandas as pd
from lxml.html import fromstring

from tqdm import tqdm
import requests

def test_proxies(proxies):
    working_proxies = []
    session = requests.Session()

    for proxy in tqdm(proxies, desc='Testing Proxies', unit='proxy'):
        ip, port = proxy.split(':')
        http_proxy = f'http://{ip}:{port}'
        https_proxy = f'https://{ip}:{port}'
        proxies = {
            'http': http_proxy,
            'https': https_proxy
        }

        try:
            response = session.get('https://www.example.com', proxies=proxies, timeout=10)
            working_proxies.append(proxy)
        except requests.exceptions.RequestException:
            pass  # Proxy connection or timeout error, ignore

    return working_proxies


def get_proxies():
    url = 'https://free-proxy-list.net/'

    response = requests.get(url)

    parser = fromstring(response.text)
    headers = [header.text_content() for header in parser.xpath('//thead/tr/th')]
    rows = parser.xpath('//tbody/tr')

    proxies = []
    for row in tqdm(rows):
        proxy = [cell.text_content() for cell in row.xpath('.//td')]
        proxies.append(proxy)

    proxies_df = pd.DataFrame(proxies, columns=headers)

    # Filter the DataFrame based on the conditions
    filtered_df = proxies_df[proxies_df['Https'] == 'yes']

    # Extract the IP address and port from the filtered DataFrame
    ip_port_list = []
    for _, row in filtered_df.iterrows():

        ip = row['IP Address']
        port = row['Port']
        ip_port = f"{ip}:{port}"
        ip_port_list.append(ip_port)

    return ip_port_list

# Get the list of proxies
proxies = get_proxies()

100%|██████████| 309/309 [00:00<00:00, 34027.51it/s]


In [43]:
# Test the proxies
working_proxies = []
while len(working_proxies) < 1:
    proxies = get_proxies()
    working_proxies = test_proxies(proxies)


Testing Proxies:  22%|██▏       | 7/32 [00:14<00:43,  1.74s/proxy]

Testing Proxies:  31%|███▏      | 10/32 [00:15<00:20,  1.06proxy/s]

In [9]:
proxy_cycle = cycle(working_proxies)

In [10]:
def pdf_downloader(doi):

    doi = doi.replace(':', '/')
    base_url = "https://sci-hub.ru"
    url = f"{base_url}/{doi}"

    proxy = next(proxy_cycle)
    proxies = {
        'http': proxy,
        'https': proxy
    }

    try:
        response = requests.get(url, proxies=proxies)
        # response = requests.get(url)
        response.raise_for_status()
    except Exception as e:
        print(f"An exception occurred: {str(e)}")

    soup = BeautifulSoup(response.content, 'html.parser')

    try:
        citation_div = soup.find('div', {'id': 'citation'})
        title = citation_div.find('i').text
        citation = citation_div.text
        titles.append(title)
        citations.append(citation)
        dois.append(doi)
    except:
        titles.append(None)
        citations.append(None)
        dois.append(doi)

    try:
        pdf_relative_url = soup.find('embed', {'type': 'application/pdf'}).get('src')
        pdf_url = urljoin(base_url, pdf_relative_url)

        pdf_response = requests.get(pdf_url, proxies=proxies)

        filename = f"data/{doi.replace('/', ':')}.pdf"

        with open(filename, "wb") as file:
            file.write(pdf_response.content)

    except Exception as e:
        return

In [11]:
with ThreadPoolExecutor(max_workers=5) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(pdf_downloader, doi) for doi in source]

    # Iterate over completed futures to check for any exceptions
    for future in tqdm(as_completed(futures), total=len(futures)):
        try:
            future.result()
        except Exception as e:
            print(f"An exception occurred: {str(e)}")

  0%|          | 1/301 [00:00<03:22,  1.48it/s]

An exception occurred: 'NoneType' object has no attribute 'get'


  1%|          | 3/301 [00:02<03:05,  1.61it/s]

An exception occurred: 'NoneType' object has no attribute 'get'
An exception occurred: 'NoneType' object has no attribute 'get'


  1%|▏         | 4/301 [00:06<09:50,  1.99s/it]

...

 97%|█████████▋| 291/301 [1:01:16<03:21, 20.19s/it]

An exception occurred: HTTPSConnectionPool(host='moscow.sci-hub.ru', port=443): Max retries exceeded with url: /2511/d38971b137b950934f94a6e68b9c625a/gornitz1991.pdf (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x162510400>, 'Connection to 158.69.185.37 timed out. (connect timeout=None)'))


 99%|█████████▊| 297/301 [1:02:26<00:34,  8.72s/it]

An exception occurred: 'NoneType' object has no attribute 'get'


 99%|█████████▉| 299/301 [1:10:03<00:28, 14.06s/it]


In [2]:
import os

folder_path = "data"  # Replace "papers" with your folder path

# Get the list of files in the folder
file_list = os.listdir(folder_path)

# Count the number of files
num_files = len(file_list)

print(f"\nThe number of files in the folder '{folder_path}' is: {num_files} out of {len(source)} initial DOIs\n")


The number of files in the folder 'data' is: 137 out of 301 initial DOIs

